In [1]:
import os
import sys
os.chdir('../')
SCRIPDIR = os.path.dirname(os.path.abspath("test.ipynb"))
sys.path.append(os.path.dirname(SCRIPDIR))
import matplotlib.pyplot as plt
import pandas as pd
from src.utils import SETUP_PATH, convert_testfile_name

In [ ]:
original_results = pd.read_csv('output/duckdb_duckdb_results.csv')
results = original_results.copy() 
# change the TESTFILE_PATH in results
results['TESTFILE_NAME'] = results['TESTFILE_PATH'].apply(lambda x: convert_testfile_name(x, 'duckdb'))    

path = SETUP_PATH['filter']
filter_df = pd.concat([pd.read_csv(f"{path}/{file}") for file in os.listdir(
    path) if file.endswith('.csv')], ignore_index=True)
filter_df[['TESTCASE_INDEX', 'CLUSTER']] = filter_df[[
    'TESTCASE_INDEX', 'CLUSTER']].astype(int)

# results_filtered = results[results['TESTFILE_NAME'].isin(filter_df['TESTFILE_NAME'])]
results_filtered = results.merge(filter_df, on='TESTFILE_NAME', how='inner')
# results_filtered = results_filtered[results_filtered
filtered = results_filtered[(results_filtered['TESTCASE_INDEX_x'] == results_filtered['TESTCASE_INDEX_y']) | (results_filtered['TESTCASE_INDEX_y'] == -1)].drop(columns=['TESTCASE_INDEX_y', 'CLUSTER', 'TESTFILE_NAME']).drop_duplicates()

In [ ]:
# sample_30 = errors.sample(n=30)
errors = original_results[original_results['IS_ERROR'] == True]
sample_100 = errors.sample(n=100, random_state=233)
sample_100['ERROR_REASON'] = ""

In [ ]:
# pre labeling
import re
sample_100.loc[sample_100.apply(lambda x: x['ERROR_MSG'].find('__TEST_DIR__') != -1, axis=1), 'ERROR_REASON'] = 1
sample_100.loc[sample_100.apply(lambda x: str(x['ACTUAL_RESULT']).find("{'key'}") != -1, axis=1), 'ERROR_REASON'] = 2
sample_100.loc[sample_100.apply(lambda x: re.search(r"\{.*\}", str(x['ACTUAL_RESULT']), re.DOTALL) is not None and re.search(r"\{.*\}", str(x['EXPECTED_RESULT']), re.DOTALL) is not None, axis=1), 'ERROR_REASON'] = 2
sample_100.loc[sample_100.apply(lambda x: re.search(r"\[.*\]", str(x['ACTUAL_RESULT']), re.DOTALL) is not None and re.search(r"\[].*\]", str(x['EXPECTED_RESULT']), re.DOTALL) is not None, axis=1), 'ERROR_REASON'] = 2

In [2]:
output_path = "output/duckdb_sample_100.csv"
output_reason_path = "output/duckdb_sample_100_reason.csv"


In [ ]:
sample_100.to_csv(output_path, index=False)


In [ ]:
cnt = 0

In [ ]:
sample = sample_100.iloc[cnt]

print("-----------I have the schema generated as follows:---------")
temp_df = results[results['TESTFILE_PATH'] == sample['TESTFILE_PATH']]
for sql in temp_df[temp_df['CASE_TYPE'] == 'Statement']['SQL']:
    print(sql + ';')
print("-----------The SQL commands is:---------")
print(sample.SQL)
print("-----------The expected result is:---------")
print(sample.EXPECTED_RESULT)
print("-----------The actual result is:---------")
print(sample.ACTUAL_RESULT)
print("-----------The error message is:---------")
print(sample.ERROR_MSG)
print(sample.LOGS_INDEX)
cnt += 1


In [ ]:
# print the sample
print(sample.TESTFILE_PATH)

# open the file and see if there's a pattern inside
with open(sample.TESTFILE_PATH) as f:
    lines = f.readlines()
    for line in lines:
        if "con1" in line:
            print(line)
            break
    else:
        print("Not found")

In [3]:
sample_100_reason = pd.read_csv(output_path)
visible_reasons = sample_100_reason[['SQL','ERROR_REASON', 'ERROR_MSG', 'ACTUAL_RESULT','EXPECTED_RESULT',  ]]
reasons_mapping = pd.read_csv('data/error_reason_manual.csv')

# add columns to sample_100_reason according to reasons_mapping
visible_reasons = pd.merge(visible_reasons, reasons_mapping, right_on='index',left_on = 'ERROR_REASON', how='inner')
visible_reasons.drop(columns=['index'], inplace=True)
visible_reasons.to_csv(output_reason_path, index=False)

In [4]:
reason_summary = visible_reasons[['SUPER_TAG', 'TAG', 'DESCRIPTION']].groupby(['SUPER_TAG', 'TAG']).count().style.to_latex()
print(reason_summary)


\begin{tabular}{llr}
 &  & DESCRIPTION \\
SUPER_TAG & TAG &  \\
\multirow[c]{2}{*}{Client} & Data Type & 66 \\
 & Numeric & 9 \\
Environment & File Paths & 20 \\
\multirow[c]{2}{*}{Misc} & Function & 1 \\
 & Parser & 4 \\
\end{tabular}

